
# Uma breve introdução ao Machine Learning: Dia 5

Gabriel Wendell Celestino Rocha

Material de um minicurso de introdução ao Machine Learning oferecido pelo [PET - Física](https://petfisica.home.blog).

O conteúdo é mantido no [GitHub]() e distribuídos sob uma [licença BSD3](https://opensource.org/licenses/BSD-3-Clause).

- [Veja a tabela de conteúdos]()

Este `Notebook` pode, opcionalmente, ser visto como uma [apresentação de slides](https://medium.com/learning-machine-learning/present-your-data-science-projects-with-jupyter-slides-75f20735eb0f). Clique [aqui]() para ver os slides online ou, para apresentar os slides localmente, use:

```Python
$ jupyter nbconvert Dia2.ipynb --to slides --post serve
```

---

### Bibliotecas necessárias

In [1]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib notebook

def opt_plot():
    # plt.style.use('dark_background')
    plt.grid(True, linestyle=':', color='0.50')
    plt.minorticks_on()
    plt.tick_params(axis='both',which='minor', direction = "in",
                        top = True,right = True, length = 5,width = 1,labelsize = 15)
    plt.tick_params(axis='both',which='major', direction = "in",
                        top = True,right = True, length = 8,width = 1,labelsize = 15)

### Versões das bibliotecas

In [2]:
%load_ext version_information
%version_information Matplotlib, Numpy, Pandas, Seaborn

Software versions
Python 3.9.8 64bit [MSC v.1929 64 bit (AMD64)]
IPython 8.0.1
OS Windows 10 10.0.19043
Matplotlib 3.5.1
Numpy 1.22.2
Pandas 1.4.0
Seaborn 0.11.2
Sat Jun 04 19:16:25 2022 Hora Padrão de Buenos Aires

#### Instalação:

```
$ pip install version_information
```

---

Carregamos apenas os módulos `scikit-learn` que precisamos:

# Baixando os dados

# Adaptando métodos lineares para resolver problemas não lineares

---

## 1. A Cura da Dimensionalidade

Já encontramos a "maldição da dimensionalidade" no contexto da redução da dimensionalidade, mas às vezes uma grande dimensionalidade pode ser uma cura. Como exemplo motivador, considere os dados 2D plotados abaixo, que claramente contêm dois clusters com formas altamente não lineares:

O gráfico acima é colorido usando os rótulos verdadeiros armazenados na coluna `y` de `circles_targets`.

Vamos usar o KMeans para ajustar esses dados como dois *clusters* e plote os resultados usando `plot_circles(labels=fit.labels_)`. 

Os *clusters* encontrados pelo KMeans não são o que queremos, mas também não surpreende, dado que o KMeans particiona as amostras com uma linha divisória simples (ou hiperplano em dimensões superiores).

Vamos criar um novo conjunto de dados chamado `circles_3d` que é uma cópia de `circles_data`, mas com um novo recurso adicionado:

$$x_{2}=x_{0}^{2}+x_{1}^{2}.$$

Pense em como esse novo recurso altera o problema de *clustering*, se for o caso.

Ao *aumentar* a dimensionalidade de nossos dados, transformamos um problema de agrupamento muito não linear em um problema linear trivial! Para ver isso, plote os dados em 3D:

Finalmente, ajustamos para 2 *clusters* KMeans em nossos novos dados `circles_3d` e plotamos os resultados, como acima.

Este é exatamente o resultado que queríamos, mas não tão surpreendente depois de ver o gráfico 3D acima.

## 2. Funções Kernel 

Existem muitas classes de problemas em que não linearidades em seus dados podem ser tratadas com métodos lineares, primeiro embutindo em uma dimensão mais alta.

A incorporação que usamos na seção anterior foi escolhida a dedo para esses dados, mas uma incorporação genérica geralmente funcionará se adicionar dimensões suficientes. Por exemplo, a função abaixo é comumente usada para incorporar recursos 2D $(x_{0},x_{1})$ em um espaço 7D:

$$\phi(x_0, x_1) = \begin{pmatrix}
x_0^2 \\
x_0 x_1 \\
x_1 x_0 \\
x_1^2 \\
\sqrt{2 c} x_0 \\
\sqrt{2 c} x_1 \\
c
\end{pmatrix}$$

Um gráfico de pares dos `circles_data` incorporados em 7D mostra que essa é uma incorporação peculiar, mas permite uma separação linear dos dois clusters (por meio de seus componentes $x^{2}_{0}$ e $x^{2}_{1}$). Também parece ineficiente, com um recurso repetido $(x_{0}x_{1})$ e outro constante $(c)$. No entanto, este é apenas o membro mais simples de uma família de embeddings onde c desempenha um papel importante na fixação da normalização relativa dos diferentes grupos de [monômios](https://en.wikipedia.org/wiki/Monomial).

A razão para escolher esta incorporação peculiar é que ela tem a seguinte propriedade muito útil:

$$\phi(X_{i})\cdot\phi(X_{j})=(X_{i}\cdot X_{j}+c)^{2},$$

onde $X_{i}$ e $X_{j}$ são amostras (*features*) arbitrárias (linhas) de nossos dados.

Primeiro, vamos verificar isso explicitamente para dados de círculos.

A razão pela qual essa propriedade é tão útil é que o RHS pode ser avaliado muito mais rápido do que o LHS e nunca exige que realmente incorporemos nossas amostras originais no espaço de dimensão superior.

As funções no espaço amostral que avaliam um produto escalar em um espaço diferente são chamadas de **funções kernel**:

$$K(X_{i},X_{j})=\phi(X_{i})\cdot\phi(X_{j}).$$

Uma função kernel é uma [medida de similaridade](https://en.wikipedia.org/wiki/Similarity_measure), pois mede a similaridade das amostras $i$ e $j$, com valor máximo para amostras idênticas e zero para amostras ortogonais. As medidas de similaridade estão relacionadas a medidas de distância (por exemplo, métricas na relatividade), mas com o comportamento oposto:

- amostras muito semelhantes: distância ~ 0, grande semelhança.
- amostras muito diferentes: grande distância, similaridade ~ 0.

A importância das funções do kernel é mais profunda do que apenas sua eficiência computacional: muitos algoritmos podem ser expressos usando apenas produtos de ponto entre amostras e, portanto, podem ser aplicados a dados incorporados em uma dimensão superior sem nunca fazer a incorporação. Esse insight é conhecido como o **truque do kernel**:

- Escolha uma função de kernel $K$.
- Escolha um algoritmo que possa ser expresso usando apenas produtos escalares.

Quando esses pré-requisitos são atendidos, o algoritmo pode ser aplicado de maneira fácil e eficiente a dados que são efetivamente impulsionados para um espaço de alta dimensão. Como vimos no exemplo acima, o principal benefício é que os dados não lineares agora podem ser analisados usando métodos lineares

Infelizmente, há um número limitado de funções de kernel adequadas $K$ (comece com o [teorema de Mercer](https://en.wikipedia.org/wiki/Mercer%27s_theorem) se você estiver interessado em aprender mais sobre o porquê disso). Já conhecemos o [kernel polinomial](https://en.wikipedia.org/wiki/Polynomial_kernel), que pode ser escrito de forma mais geral como:

$$K(X_{i},X_{j})=(\gamma X_{i}\cdot X_{j}+c)^{d},$$

onde $\gamma$, $c$ e $d$ são todos hiperparâmetros (nosso exemplo anterior usou $\gamma=1$, $c=1$ e $d=2$). O módulo `metric.pairwise` do `sklearn` pode calcular a matriz de todos os produtos escalares de amostra possíveis para este e outros kernels, por exemplo:

Outros kernels populares são o [kernel sigmoid](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.sigmoid_kernel.html):

$$\boxed{K(X_{i},X_{j})=\tanh{(\gamma X_{i}\cdot X_{j}+c)}},$$

Além disso, o [kernel da função de base radial (rbf)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.rbf_kernel.html) (cujo embutimento é de dimensão infinita devido à expansão em série infinita de $e^{−x}$):

$$\boxed{K(X_{i},X_{j})=\exp{\Bigg(-\gamma|X_{i}-X_{j}|^{2}\Bigg)}}.$$

## 3. Kernel PCA

Como um exemplo do **truque do kernel**, o algoritmo PCA pode ser adaptado para usar apenas produtos de ponto para projetar cada amostra nos autovetores de dimensão superior. O [algoritmo KernelPCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html) resultante é usado exatamente como os métodos de decomposição linear, mas com alguns hiperparâmetros adicionais, por exemplo.

```Python
fit = decomposition.KernelPCA(n_components=d, kernel='poly', gamma=1., degree=2).fit(X)
Y = fit.transform(X)
```

Uma limitação do truque do kernel para PCA é que as amostras originais não podem ser reconstruídas usando apenas produtos de ponto, de modo que a reconstrução das variáveis latentes se torna um novo problema desafiador que requer uma abordagem de [aprendizado de máquina supervisionada separada](http://papers.nips.cc/paper/2417-learning-to-find-pre-images.pdf). Felizmente, a implementação do `sklearn` cuida de tudo isso para você:

```Python
fit = decomposition.KernelPCA(n_components=d, kernel='poly', gamma=1., degree=2, inverse_transform=True).fit(X)
Y = fit.transform(X)
reconstructed = fit.inverse_transform(Y)
```

Usaremos a seguinte função para demonstrar o método KernelPCA, que permite definir o hiperparâmetro $\gamma$:

O resultado é espetacular: nossos dados não lineares são completamente linearizados quando transformados em um espaço variável latente 2D. Observe que, neste exemplo, não estamos realizando nenhuma redução geral de dimensionalidade: começamos com 2 recursos, expandimos implicitamente para um número infinito de recursos usando o kernel RBF, depois reduzimos para duas variáveis latentes.

Os resultados acima são bastante sensíveis à escolha dos hiperparâmetros. Para explorar isso, vamos executar novamente `kpca_demo` com diferentes valores de $\gamma$.

Os resultados são bem ajustados e pequenas variações em $\gamma$ podem destruir a separação linear. A sensibilidade a $\gamma$ não é muito surpreendente, pois é um parâmetro da função kernel. No entanto, os resultados do KernelPCA também podem mudar drasticamente com uma pequena alteração nos dados de entrada. Consulte este [problema do github](https://github.com/scikit-learn/scikit-learn/issues/10530) para obter detalhes.

## 4. Incorporação linear local

O **truque do kernel** não é a única maneira de aproveitar métodos lineares para problemas não lineares. Para o nosso próximo exemplo, consideramos a **incorporação linear local** (LLE), que é um tipo de "aprendizagem múltipla", ou seja, um método de redução de dimensionalidade para dados em uma variedade não linear.

Primeiro vamos ver alguns dados 3D que são claramente 2D, mas requerem uma decomposição não linear.

O pairplot é confuso até que você veja o seguinte gráfico onde, além disso, cada ponto é colorido de acordo com sua coordenada 1D verdadeira ao longo da direção principal do coletor (armazenado na coluna `y` de `ess_targets`):

O LLE aproveita o fato de que a variedade é "plana" na vizinhança de cada amostra, portanto pode ser descrita localmente com uma aproximação linear. Construímos uma aproximação linear local para uma amostra $\overrightarrow{X_{i}}$ como:

$$\boxed{\overrightarrow{X_{i}}\approxeq\sum_{j\neq i}W_{ij}\overrightarrow{X_{j}}}$$

em relação aos pesos $n\times N$ em $W$.

O principal insight do LLE é que, uma vez que um conjunto adequado de pesos $W$ tenha sido encontrado:

- eles descrevem completamente a geometria local do coletor, e
- esta geometria pode então ser *transferida* para outro espaço (menor) de variáveis latentes.

A maneira como isso funciona é minimizar uma segunda função objetivo muito semelhante

$$\boxed{\sum_{i}\Bigg|\overrightarrow{Y_{i}}-\sum_{j\neq i}W_{ij}\overrightarrow{X_{j}}\Bigg|^{2}}$$

onde cada amostra $X_{i}$ tem um $Y_{i}$ correspondente, mas estes podem ter dimensões completamente diferentes! Observe que, embora as funções de metas pareçam semelhantes, os parâmetros que minimizamos são diferentes a cada vez:

- Primeiro minimizamos em relação aos elementos de $W$, com os dados de entrada $X$ fixos.

- Em seguida, minimizamos em relação às variáveis latentes $Y_{i}$, com a matriz de pesos $W$ fixa.

Este método foi descoberto em 2000 e o [artigo original](https://www.science.org/doi/10.1126/science.290.5500.2323) é bastante acessível.

O [método LLE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.LocallyLinearEmbedding.html) reside no módulo `sklearn` chamado `manifold` e segue o padrão de chamada usual, com dois hiperparâmetros significativos:

- O número de vizinhos mais próximos a serem usados para calcular $W$.

- O número de variáveis latentes (componentes) a serem usadas em $Y$.

Para obter resultados reproduzíveis, você também deve passar um RandomState.

Depois de projetar no espaço latente, descobrimos que a forma em S foi efetivamente achatada, embora não em um belo retângulo:

Comparamos isso com o que um PCA linear encontra:

Ou um KernelPCA:

A classe `sklearn` LLE também fornece algumas variantes de LLE que podem ter um desempenho ainda melhor nesse problema, por exemplo (observe os `n_neighbors` maiores necessários - outro exemplo de ajuste fino):

![image](https://c.tenor.com/hEOM8E4epvgAAAAC/hahaha-thats-all-folks.gif)

---